Hilbert 4^6 = 4096 different areas

Fixed length: 224 US

GASF + GADF + BLACK

In [ ]:
#Importing useful python packages

import numpy as np
import pandas as pd
import math
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib.lines import Line2D
import yaml
from IPython.display import display
try:
    from yaml import CLoader as Loader, CDumper as Dumper
except ImportError:
    from yaml import Loader, Dumper

from sklearn.model_selection import ParameterGrid

pd.options.display.max_rows = 9999
pd.options.display.max_columns = 9999

#Initializing the random state variable rs so that the results could be replicated

rs = 42
np.random.seed(42)

In [ ]:
from Hilbert_function import hilbIndex
from US_function import uniform_Sampling
from Duplicate_function import duplicate

In [ ]:
#Loading the dataset and storing it in the variable df (as a dataframe)

my_path = "midimost_with_class2.csv"

In [ ]:
df_6 = pd.read_csv(my_path, sep = ',', low_memory = False)

In [ ]:
print(df_6['class2'].unique())

In [ ]:
df_stat_6 = df_6.groupby(['vehicle_id'], as_index = False).agg(
    x_min = ('vehicle_x','min'),
    x_max = ('vehicle_x','max'),
    y_min = ('vehicle_y','min'),
    y_max = ('vehicle_y','max'),
    n_lines_x = ('vehicle_x','count'),
    n_lines_y = ('vehicle_y', 'count')
)

print(sum(df_stat_6['n_lines_x'] != df_stat_6['n_lines_y']))

x_glob_min_6 = df_stat_6['x_min'].min()
x_glob_max_6 = df_stat_6['x_max'].max()

y_glob_min_6 = df_stat_6['y_min'].min()
y_glob_max_6 = df_stat_6['y_max'].max()

df_new_6 = df_6.copy()
vehicle_id_to_remove = df_stat_6[df_stat_6['n_lines_x'] < 4]['vehicle_id']
df_new_6 = df_new_6[~ df_new_6['vehicle_id'].isin(vehicle_id_to_remove)]


df_new_6['vehicle_x'] = (df_new_6['vehicle_x'] - x_glob_min_6) / (x_glob_max_6 - x_glob_min_6)
df_new_6['vehicle_y'] = (df_new_6['vehicle_y'] - y_glob_min_6) / (y_glob_max_6 - y_glob_min_6)

In [ ]:
df_new_6.describe()

In [ ]:
eps = 0.00095

df_new_6['Hilbert'] = [hilbIndex(x, y, eps) for x,y in zip(df_new_6['vehicle_x'], df_new_6['vehicle_y'])]

In [ ]:
grouped_df_6 = df_new_6.groupby('vehicle_id').agg(
    labels = ('class2', list),
    Hilbert = ('Hilbert', list)
).reset_index()

grouped_df_6['label'] = grouped_df_6['labels'].apply(lambda x: x[0])

In [ ]:
grouped_df_6['squeezed_Hilbert'] = grouped_df_6['Hilbert'].apply(uniform_Sampling, final_length = 224)
grouped_df_6['squeezed_Hilbert'] = grouped_df_6['squeezed_Hilbert'].apply(duplicate, K = 224)

In [ ]:
labels_6 = grouped_df_6['label'].copy()

In [ ]:
X_6 = grouped_df_6['squeezed_Hilbert'].copy()

In [ ]:
class_, counts_ = np.unique(labels_6, return_counts = True)
fig, ax = plt.subplots(1, 1, figsize = (6, 3))
plt.bar(class_, counts_, color = 'lightgreen')
ax.set_ylabel("Count", fontsize = 12)
ax.set_xlabel("Category", fontsize = 12)
ax.set_xticks(class_)
ax.set_xticklabels(class_, rotation = 45, ha = "right")
plt.grid(color = 'lightsteelblue', linestyle = '-', linewidth = 0.2)
plt.tight_layout()
plt.show()

In [ ]:
X_6.shape

In [ ]:
counts_

In [ ]:
X_6 = np.vstack(np.array(X_6))

In [ ]:
from pyts.image import GramianAngularField, MarkovTransitionField

In [ ]:
gaf_s = GramianAngularField(method = 'summation', sample_range = (0, 1))
gaf_d = GramianAngularField(method = 'difference', sample_range = (0, 1))

In [ ]:
X_gaf_s_6 = gaf_s.fit_transform(X_6)
X_gaf_d_6 = gaf_d.fit_transform(X_6)

In [ ]:
R_6 = ((X_gaf_s_6 - X_gaf_s_6.min()) / (X_gaf_s_6.max() - X_gaf_s_6.min())) * 255
G_6 = ((X_gaf_d_6 - X_gaf_d_6.min()) / (X_gaf_d_6.max() - X_gaf_d_6.min())) * 255
B_6 = X_0_6 = np.full((X_gaf_s_6.shape[0], 224, 224), 0)
X_color_new_6 = np.stack((R_6, G_6, B_6), axis = -1)

In [ ]:
import os
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split

In [ ]:
y_6 = np.array(labels_6)

In [ ]:
x_train_6, x_val_test_6, y_train_6, y_val_test_6 = train_test_split(X_color_new_6, y_6, test_size = 0.4, random_state = rs, stratify = y_6)
x_val_6, x_test_6, y_val_6, y_test_6 = train_test_split(x_val_test_6, y_val_test_6, test_size = 0.5, random_state = rs, stratify = y_val_test_6)

In [ ]:
os.makedirs("Data_35/train/pedestrian", exist_ok = False)
os.makedirs("Data_35/val/pedestrian", exist_ok = False)
os.makedirs("Data_35/test/pedestrian", exist_ok = False)

os.makedirs("Data_35/train/bus", exist_ok = False)
os.makedirs("Data_35/val/bus", exist_ok = False)
os.makedirs("Data_35/test/bus", exist_ok = False)

os.makedirs("Data_35/train/motorcycle", exist_ok = False)
os.makedirs("Data_35/val/motorcycle", exist_ok = False)
os.makedirs("Data_35/test/motorcycle", exist_ok = False)

os.makedirs("Data_35/train/bycicle", exist_ok = False)
os.makedirs("Data_35/val/bycicle", exist_ok = False)
os.makedirs("Data_35/test/bycicle", exist_ok = False)

os.makedirs("Data_35/train/private_car", exist_ok = False)
os.makedirs("Data_35/val/private_car", exist_ok = False)
os.makedirs("Data_35/test/private_car", exist_ok = False)

os.makedirs("Data_35/train/taxi_or_uber", exist_ok = False)
os.makedirs("Data_35/val/taxi_or_uber", exist_ok = False)
os.makedirs("Data_35/test/taxi_or_uber", exist_ok = False)

In [ ]:
def save_images(images, labels, partition, directory):
    for idx, label in enumerate(labels):
        I = Image.fromarray(images[idx].astype(np.uint8))
        
        folder = os.path.join(directory, partition)
        folder = os.path.join(folder, label)

        filename = f"{label}.{idx}.png"  
        I.save(os.path.join(folder, filename), format = 'TIFF')

In [ ]:
save_images(x_train_6, y_train_6, "train", "Data_35")
save_images(x_val_6, y_val_6, "val", "Data_35")
save_images(x_test_6, y_test_6, "test", "Data_35")